In [1]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

2022-06-01 09:52:14,701 dwave.cloud INFO MainThread Log level for 'dwave.cloud' namespace set to 0
2022-06-01 09:52:14 INFO     Log level for 'dwave.cloud' namespace set to 0
2022-06-01 09:52:15 INFO     Found credentials in shared credentials file: ~/.aws/credentials


# Step 1: Prepare Data

In this part, we load the raw molecule data for experiment.
The [117 ligand](http://www.rcsb.org/ligand/117) was 
put in the repository. We assign the relative 
path to **raw_path**.
The **s3_bucket** and **prefix** are used to store the 
optimization results. We can use the one created with the 
cloudformation for convenience.

In [2]:
# initial parameters for experiment data
s3_bucket = f"amazon-braket-1a222675c751" # change to the name of bucket created in your deployment
prefix = "annealer-experiment" # the name of the folder in the bucket

# raw_path = './molecule-data/117_ideal.mol2' # the mol2 file for this experiment
raw_path = '/Users/aoyuzhan/Workplace/SolutionCenter/initiative/qc/hcls/drug-discovery/AWS-gcr-qc-life-science/source/src/molecule-unfolding/molecule-data/Aspirin.mol2'

In [3]:
mol_data = MoleculeData(raw_path, 'qmu')

data_path = mol_data.save("latest")

num_rotation_bond = mol_data.bond_graph.rb_num
print(f"You have loaded the raw molecule data and saved as {data_path}. \n\
This molecule has {num_rotation_bond} rotable bond")

2022-06-01 09:52:16 INFO     parse mol2 file!
2022-06-01 09:52:16 INFO     finish save qmu_Aspirin_data_latest.pickle


You have loaded the raw molecule data and saved as ./qmu_Aspirin_data_latest.pickle. 
This molecule has 4 rotable bond


After running this block, the processed data 
will be saved as **qmu_117_ideal_data_latest.pickle**
and **data_path** will be updated. We can see that this 
molecule has 23 rotatable bonds.

# Step 2: Build Model

In this part, we build the Quadratic Unconstrained 
Binary Optimization (QUBO) model for molecular unfolding.

First, we set the following parameters and 
initialize the QMUQUBO object. 

<center>

| Parameter | Description | Value |
|--- |--- |--- |
|A | penalty scalar |300|
|hubo_qubo_val | energy penalty of make_quadratic() |200|
|M | number of torsions for molecular unfolding| [1, max number of rotatable bonds] |
|D| angle precision of rotation| 8|
|method| the method of building model| 'pre-calc': calculate the score in advance|

 </center>

We use the 'pre-calc' method 
to build the model. This molecule has 23 rotatable bonds and 
we only test 2 of them, so we set the **M** to 2. And we want 
the angle to become $45^o$, so we set the **D** to 8 
(i.e., $8=360^o/45^o$). The **A** and **hubo_qubo_val** are 
test from experiments. 

In [4]:
# initial the QMUQUBO object
init_param = {}
method = ['pre-calc', 'symbolic']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    elif mt == 'symbolic':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'strength']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

2022-06-01 09:52:20 INFO     initial pre-calculate for constructing molecule QUBO
2022-06-01 09:52:20 INFO     initial symbolic calculate for constructing molecule QUBO


In [5]:
# set the parameters for model
model_param = {}
# parameters
num_rotation_bond = mol_data.bond_graph.rb_num

method = 'pre-calc'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [3,4]
model_param[method]['D'] = [8]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]

method = 'symbolic'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [3,4]
model_param[method]['D'] = [8]
model_param[method]['A'] = [300]
model_param[method]['strength'] = [200]

qmu_qubo.build_model(**model_param)

2022-06-01 09:52:22 INFO     debug for pre-calc 1
2022-06-01 09:52:22 INFO     debug for pre-calc 2
2022-06-01 09:52:22 INFO     debug for pre-calc 3
2022-06-01 09:52:22 INFO     debug for pre-calc 4
2022-06-01 09:52:22 INFO     debug for pre-calc 5
2022-06-01 09:52:22 INFO     debug for pre-calc 6
2022-06-01 09:52:22 INFO     Construct model for M:3,D:8,A:300,hubo_qubo_val:200 0.007141053676605225 min
2022-06-01 09:52:22 INFO     debug for pre-calc 1
2022-06-01 09:52:24 INFO     debug for pre-calc 2
2022-06-01 09:52:24 INFO     debug for pre-calc 3
2022-06-01 09:52:24 INFO     debug for pre-calc 4
2022-06-01 09:52:24 INFO     debug for pre-calc 5
2022-06-01 09:52:24 INFO     debug for pre-calc 6
2022-06-01 09:52:24 INFO     Construct model for M:4,D:8,A:300,hubo_qubo_val:200 0.03230765263239543 min
2022-06-01 09:52:24 INFO     debug for symbolic 1
2022-06-01 09:52:24 INFO     Rotables influence set(ris) group 4+5
2022-06-01 09:52:24 INFO     update hubo 1
2022-06-01 09:52:24 INFO     

We can use the following method to check the properties of 
model. This way, we can build many models conveniently. 
After that, we save the model and update the value of 
**model_path**.

In [6]:
# describe the model parameters
model_info = qmu_qubo.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {3, 4}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}
INFO:root:method: symbolic
INFO:root:param: M, value set()
INFO:root:param: D, value set()
INFO:root:param: A, value set()
INFO:root:param: strength, value set()


In [ ]:
# save the model
model_path = qmu_qubo.save("latest")

print(f"You have built the QUBO model and saved it as {model_path}")

INFO:root:finish save qmu_Aspirin_model_latest.pickle


You have built the QUBO model and saved it as ./qmu_Aspirin_model_latest.pickle


In [7]:
M = 2
D = 8
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qmu_qubo.mol_data.bond_graph.sort_ris_data['2']

{'4+5': {'metrics': '4+5',
  'f_0_set': {'2'},
  'f_1_set': {'10',
   '11',
   '12',
   '13',
   '17',
   '18',
   '19',
   '20',
   '6',
   '7',
   '8',
   '9'},
  'avg_bc_num': 0.5146198830409356,
  'rb_count_num': 1},
 '2+4': {'metrics': '2+4',
  'f_0_set': {'1', '14', '15', '16', '3'},
  'f_1_set': {'5'},
  'avg_bc_num': 0.4444444444444444,
  'rb_count_num': 1},
 '4+5,2+4': {'metrics': '4+5',
  'f_0_set': {'1', '14', '15', '16', '3'},
  'f_1_set': {'10',
   '11',
   '12',
   '13',
   '17',
   '18',
   '19',
   '20',
   '6',
   '7',
   '8',
   '9'},
  'avg_bc_num': 0.47953216374269003,
  'rb_count_num': 2}}

In [8]:
mol_data = qmu_qubo.mol_data
atom_pos_data = qmu_qubo.atom_pos_data
atom_pos_data

{'1': {'pts': [-1.4637, 1.3943, 1.2946],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '2': {'pts': [-1.3545, -0.1059, 1.201],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '3': {'pts': [-1.4493, -0.7819, 2.198],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.52},
 '4': {'pts': [-1.1519, -0.6914, 0.0093],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.52},
 '5': {'pts': [-0.9578, -2.0342, 0.0079],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '6': {'pts': [0.327, -2.5523, 0.0594],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '7': {'pts': [0.5232, -3.9195, 0.0578],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '8': {'pts': [-0.5586, -4.7832, 0.005],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '9': {'pts': [-1.843, -4.2868, -0.0466],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},
 '10': {'pts': [-0.3626852691942779, -2.49883534441342, 0.12424582077927981],
  'idx': ([0, 0, 0], [0, 0, 0]),
  'vdw-radius': 1.7},

In [20]:
from pyqubo import Array
from pyqubo import Num
from pyqubo import Binary
import math
import numpy as np

def rot_mat_value(vec, orig, c, s):

    x_vector = vec[0]
    y_vector = vec[1]
    z_vector = vec[2]
    x_orig = orig[0]
    y_orig = orig[1]
    z_orig = orig[2]

    u = x_orig - x_vector
    v = y_orig - y_vector
    w = z_orig - z_vector

    u2 = u * u
    v2 = v * v
    w2 = w * w

    l2 = u * u + v * v + w * w

    l = math.sqrt(l2)

    cost = c

    print(f"symbolic c is {c}")

    sint = s

    print(f"symbolic s is {s}")

    one_minus_cost = 1 - c

    matrix_list =[]
    row_list = []
    row_list.append((u2 + (v2 + w2) * cost)/l2)
    row_list.append((u * v * one_minus_cost - w * l * sint)/l2)
    row_list.append((u * w * one_minus_cost + v * l * sint)/l2)
    row_list.append(((x_orig * (v2 + w2) - u * (y_orig * v + z_orig * w)) * one_minus_cost + (y_orig * w - z_orig * v) * l * sint) / l2)
    # row_list = [round(num, 4) for num in row_list]

    matrix_list.append(row_list)

    row_list = []
    row_list.append((u * v * one_minus_cost + w * l * sint)/l2)
    row_list.append((v2 + (u2 + w2) * cost)/l2)
    row_list.append((v * w * one_minus_cost - u * l * sint)/l2)
    row_list.append(((y_orig * (u2 + w2) - v * (x_orig * u + z_orig * w)) * one_minus_cost + (z_orig * u - x_orig * w) * l * sint) / l2)
    # row_list = [round(num, 4) for num in row_list]

    matrix_list.append(row_list)

    row_list = []
    row_list.append((u * w * one_minus_cost - v * l * sint)/l2)
    row_list.append((v * w * one_minus_cost + u * l * sint)/l2)
    row_list.append((w2 + (u2 + v2) * cost)/l2)
    row_list.append(((z_orig * (u2 + v2) - w * (x_orig * u + y_orig * v)) * one_minus_cost + (x_orig * v - y_orig * u) * l * sint) / l2)
    # row_list = [round(num, 4) for num in row_list]

    matrix_list.append(row_list)

    row_list = []
    row_list.append(0)
    row_list.append(0)
    row_list.append(0)
    row_list.append(1)
    # row_list = [round(num, 4) for num in row_list]

    matrix_list.append(row_list)

    # np_matrix = np.array(matrix_list)
    rot_binary = Array(matrix_list)

    return rot_binary, matrix_list



In [21]:
# initial variables
print(f"var name is {mol_data.bond_graph.rb_name}")

D = 2

var = {} 
var_rb_map = {}
rb_var_map = {}

for m, name in enumerate(mol_data.bond_graph.rb_name):
    x_d = {}
    var_rb_map[str(m+1)] = name
    rb_var_map[str(name)] = str(m+1)
    d = 0
    exec (f"x_{m+1}_{d+1} = Binary(f'x_{m+1}_{d+1}')") 
    exec (f"x_d[str({d+1})] = x_{m+1}_{d+1}") 

    exec (f"s_value = math.sin(2*math.pi*{d}/({D}))")
    exec (f"x_{m+1}_s_1 = x_{m+1}_{d+1} * s_value")
    exec (f"s_value = math.cos(2*math.pi*{d}/({D}))")
    exec (f"x_{m+1}_c_1 = x_{m+1}_{d+1} * s_value")

    for d in range(1, D):
        exec (f"x_{m+1}_{d+1} = Binary(f'x_{m+1}_{d+1}')") 
        exec (f"x_d[str({d+1})] = x_{m+1}_{d+1}") 

        exec (f"s_value = math.sin(2*math.pi*{d}/({D}))")
        exec (f"x_{m+1}_s_1 = x_{m+1}_s_1 + x_{m+1}_{d+1} * s_value")
        exec (f"s_value = math.cos(2*math.pi*{d}/({D}))")
        exec (f"x_{m+1}_c_1 = x_{m+1}_c_1 + x_{m+1}_{d+1} * s_value")
    exec(f"x_d['s_1'] = x_{m+1}_s_1")
    exec(f"x_d['c_1'] = x_{m+1}_c_1")
    var[str(m+1)] = x_d

var name is ['1+2', '2+4', '4+5', '10+11']


In [22]:

var

{'1': {'1': Binary(x_1_1),
  '2': Binary(x_1_2),
  's_1': (Num(0.000000)+Binary(x_1_2)*Num(0.000000)),
  'c_1': (Binary(x_1_1)+Binary(x_1_2)*Num(-1.000000))},
 '2': {'1': Binary(x_2_1),
  '2': Binary(x_2_2),
  's_1': (Num(0.000000)+Binary(x_2_2)*Num(0.000000)),
  'c_1': (Binary(x_2_1)+Binary(x_2_2)*Num(-1.000000))},
 '3': {'1': Binary(x_3_1),
  '2': Binary(x_3_2),
  's_1': (Num(0.000000)+Binary(x_3_2)*Num(0.000000)),
  'c_1': (Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))},
 '4': {'1': Binary(x_4_1),
  '2': Binary(x_4_2),
  's_1': (Num(0.000000)+Binary(x_4_2)*Num(0.000000)),
  'c_1': (Binary(x_4_1)+Binary(x_4_2)*Num(-1.000000))}}

In [23]:
import logging
M = 2
A = 300
shots = 1000

hubo_constraints = Num(0)
hubo_distance = Num(0)

qmu_qubo._init_mol_file()
atom_pos_data = qmu_qubo.atom_pos_data

def update_constraint(ris, hubo_constraints):
    # implement variable for one torsion
    for d1 in range(D):
        var_1 = var[rb_var_map[ris]][str(d1+1)]
        for d2 in range(D):
            var_2 = var[rb_var_map[ris]][str(d2+1)]
            if var_1 == var_2:
                hubo_constraints = hubo_constraints + var_1 * var_1 * -A
            else:
                hubo_constraints = hubo_constraints + var_1 * var_2 * 2 * A
    
    return hubo_constraints

def update_hubo(torsion_group, hubo_distance, tor_points):
    rot_mat = None

    for tor in torsion_group:
        c = var[rb_var_map[tor]]['c_1']
        s = var[rb_var_map[tor]]['s_1']
        start_pts = atom_pos_data[tor.split('+')[0]]
        end_pts = atom_pos_data[tor.split('+')[1]]
        if rot_mat is None:
            rot_mat, _ = rot_mat_value(start_pts['pts'], end_pts['pts'], c, s)
        else:
            new_mat, _ = rot_mat_value(start_pts['pts'], end_pts['pts'], c, s)
            rot_mat = new_mat.matmul(rot_mat)
        logging.info(f"rot mat is {rot_mat}")
    # update points distance
    tor_points_f_0_set = tor_points['f_0_set']
    tor_points_f_1_set = tor_points['f_1_set']

    for f_0 in tor_points_f_0_set:
        for f_1 in tor_points_f_1_set:
            a_pos = atom_pos_data[f_0]['pts']
            b_pos = atom_pos_data[f_1]['pts']
            a_homo_pos = Array([[a_pos[0]], [a_pos[1]], [a_pos[2]], [1]])
            b_homo_pos = Array([[b_pos[0]], [b_pos[1]], [b_pos[2]], [1]])
            result_array = a_homo_pos - rot_mat.matmul(b_homo_pos)
            distance = result_array.T.matmul(result_array)
            hubo_distance = hubo_distance - distance[0][0]
    
    return hubo_distance

for ris in mol_data.bond_graph.sort_ris_data[str(M)].keys():
    start = time.time()
    print(f"Rotables influence set(ris) group {ris}")
    end = time.time()
    torsion_group = ris.split(",")
    if len(torsion_group) == 1:
        # update constraint
        hubo_constraints = update_constraint(ris, hubo_constraints)

    ris_data = mol_data.bond_graph.sort_ris_data[str(M)][ris]
    hubo_distance = update_hubo(torsion_group, hubo_distance, ris_data)
    break

INFO:root:rot mat is Array([[((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(1.803114)+Num(0.037675))*Num(0.543245), (((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(-1.000000)+Num(1.000000))*Num(-0.260637)+(Num(0.000000)+Binary(x_3_2)*Num(0.000000))*Num(-0.001899)*Num(-1.000000))*Num(0.543245), (((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(-1.000000)+Num(1.000000))*Num(-0.000272)+(Num(0.000000)+Binary(x_3_2)*Num(0.000000))*Num(-1.821853))*Num(0.543245), (((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(-1.000000)+Num(1.000000))*Num(-2.257209)+(Num(0.000000)+Binary(x_3_2)*Num(0.000000))*Num(0.018257))*Num(0.543245)],
       [(((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(-1.000000)+Num(1.000000))*Num(-0.260637)+(Num(0.000000)+Binary(x_3_2)*Num(0.000000))*Num(-0.001899))*Num(0.543245), ((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(0.037677)+Num(1.803112))*Num(0.543245), (((Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))*Num(-1.000000)+Num(1.000000))*Num(0.001880)+(Num(0.00

Rotables influence set(ris) group 4+5
symbolic c is (Binary(x_3_1)+Binary(x_3_2)*Num(-1.000000))
symbolic s is (Num(0.000000)+Binary(x_3_2)*Num(0.000000))


In [47]:
strength = 100000

hubo_model = hubo_distance + hubo_constraints

model = hubo_model.compile(strength)

# model = hubo_model.compile()

bqm = model.to_bqm()

In [48]:
import neal 
shots = 10000
sampler = neal.SimulatedAnnealingSampler()

sampleset = sampler.sample(bqm, num_reads=shots)

decode_samples = model.decode_sampleset(sampleset)

In [49]:
# print("default strength")
analysis_result(decode_samples)

annealing energy == -621354.8396579397 with A = 300 shots = 10000 strength = 100000
x_1_5 == 1


In [44]:
def analysis_result(decode_samples):
    samples = min(decode_samples, key=lambda x: x.energy)

    type(samples)

    result = samples.sample

    print(f"annealing energy == {samples.energy} with A = {A} shots = {shots} strength = {strength}")

    for k,v in result.items():
        if k.split('_')[0] == 'x' and v == 1:
            print(f"{k} == 1")

In [157]:
r45_vector = (-1.1519, -0.6914, 0.0093)
r45_orig = (-0.9578, -2.0342, 0.0079)
r45_rot_binary, r45_mat = rot_mat(r45_vector, r45_orig)
# r45_orig = (-1.1519, -0.6914, 0.0093)
# r45_vector = (-0.9578, -2.0342, 0.0079)
# r45_rot_binary, r45_mat = rot_mat(r45_vector, r45_orig)


In [150]:
r24_r45_binary = r24_rot_binary.matmul(r45_rot_binary)

# Step 3: Optimize Configuration

In this part, we use SA and QA to find the optimized configuration of molecular unfolding.
At first, we load the model file using **QMUQUBO** object

In [8]:
qmu_qubo_optimize = QMUQUBO.load(model_path)

In [9]:
model_info = qmu_qubo_optimize.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {1, 2, 3, 4}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


We can see the parameters of this model, with M equaling 1,2 or 3, D equaling 8, 
A equaling 300 and hubo_qubo_val equaling 200. 
Actually, we can contain multiple models in this file just 
by giving multiple values for one parameter when creating models.

Actually, we can contain multiple models in this file just 
by giving multiple values for one parameter when creating models.
Then, we need use **model_name** to get the model for experiments.

In [50]:
# get the model you want to optimize
M = 3
D = 8
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

We can see that we want to carry out experiment with the QUBO model with M equaling 2.
 After that, we set the parameters for optimization.

| Parameter | Description | Value |
|--- |--- |--- |
|method | annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
|shots| number of reads, refer to [dwave-sa](https://docs.ocean.dwavesys.com/projects/neal/en/latest/reference/generated/neal.sampler.SimulatedAnnealingSampler.sample.html#neal.sampler.SimulatedAnnealingSampler.sample) and [dwave-qa](https://amazon-braket-ocean-plugin-python.readthedocs.io/en/latest/_apidoc/braket.ocean_plugin.braket_sampler.html) for details |1 to 10,000|
|bucket | the s3 bucket to store your results | - |
|prefix | the name of the folder in your s3 bucket | - |
|device | the arn name to run your quantum annealing| 'arn:aws:braket:::device/qpu/d-wave/Advantage_system4' <br> 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6'|

Then, we can run the SA for this problem:

In [51]:
method = 'neal-sa'

optimizer_param = {}
optimizer_param['shots'] = 10000

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use neal simulated annealer (c++) from dimod


In [52]:
sa_optimize_result = sa_optimizer.fit()

INFO:root:fit() ...
INFO:root:neal-sa save to local
INFO:root:finish save neal-sa_result.pickle


We can tell that we set the number of shots for SA to 1000. 
The result is saved as the local file **./sa_result.pickle.**
Alternatively, we can use QA to solve this problem:

In [316]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1000
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:root:use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


In this QA, we set the number of shots to 1000 and 
choose the 
[Advantage_System4.1](https://docs.dwavesys.com/docs/latest/doc_physical_properties.html)
as the QPU. In addition, the results are saved to your bucket automatically and you 
can get the task id for future process. 

In [ ]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

In [318]:
qa_task_id = qa_optimizer.get_task_id()
print(f"task id is {qa_task_id}")

task id is f4081e34-c785-4ba7-9718-cbea1a308596


Finally, we can compare the execution time between SA and QA :

In [26]:
print(f"dwave-sa run time {sa_optimize_result['time']}")
print(f"dwave-qa run time {qa_optimize_result['time']}")

dwave-sa run time 129.5549328327179
dwave-qa run time 11.894113779067993


We can tell from the image that SA needs more than 150 seconds 
and QA needs more than 5 seconds to find 
solution.

We sometimes get the best result that occurs only once.

![OneTimeQA](../../../docs/en/images/one-time-qa.png)

This does not always indicate an error. It is actually the characteristic of the problem or how the problem 
is formulated. Because we have different linear and quadratic terms that vary by many orders of magnitude. If we 
set change value of A to some smaller number, like 10 or 100, more occurrences of the best answer will be observed. 
However, these answers usually break the constraints. For more information about this phenomenon, please refer to this 
[Link](https://support.dwavesys.com/hc/en-us/community/posts/1500000698522-Number-of-occurrences-?input_string=number%20occurance).

# Step 4: PostProcess Result

In this part, we post process the optimizing results for evaluation and visualization.
At first, we prepare the following parameters:

| Parameter | Description | Value |
|--- |--- |--- |
|method | annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
|raw_path| the path for the original molecule file| './molecule-data/117_ideal.mol2' in this example |
|data_path| the path for the processed molecule file| './qmu_117_ideal_data_latest.mol2' in this example |
|bucket | the s3 bucket to store your results | - |
|prefix | the name of the folder in your s3 bucket | - |
|task_id | the id for your quantum annealing task| '2b5a3b05-1a0e-443a-852c-4ec422a10e59' in this example |

Then we can run the post-process using **ResultParser** object for SA:

In [53]:
method = "neal-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer neal-sa raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for neal-sa: 
     local time is 9.917102098464966


In [54]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_result_filepath, sa_result_json = sa_process_result.save_mol_file(f"{timestamp}")

print(f"result path is {sa_result_filepath}, and result optimization file path is {sa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'2': ['1', '3', '4', '8', '2', '6', '5', '7'], '3': ['1', '7', '4', '3', '6', '5', '8', '2'], '1': ['8', '4', '3', '2', '7', '1']} var_dict_list [{'2': '1', '3': '1', '1': '8'}, {'2': '1', '3': '1', '1': '4'}, {'2': '1', '3': '1', '1': '3'}, {'2': '1', '3': '1', '1': '2'}, {'2': '1', '3': '1', '1': '7'}, {'2': '1', '3': '1', '1': '1'}, {'2': '1', '3': '7', '1': '8'}, {'2': '1', '3': '7', '1': '4'}, {'2': '1', '3': '7', '1': '3'}, {'2': '1', '3': '7', '1': '2'}, {'2': '1', '3': '7', '1': '7'}, {'2': '1', '3': '7', '1': '1'}]
INFO:root:chosen var {'x_2_1', 'x_3_1', 'x_3_7', 'x_3_4', 'x_2_3', 'x_3_3', 'x_2_4', 'x_2_8', 'x_1_8', 'x_1_4', 'x_3_6', 'x_2_2', 'x_2_6', 'x_3_5', 'x_1_3', 'x_3_8', 'x_3_2', 'x_2_5', 'x_1_2', 'x_1_7', 'x_2_7', 'x_1_1'}
INFO:root:tor list {'X_3_1', 'X_1_8', 'X_2_1'}
INFO:root:initial 2231.75575498
INFO:root:optimize 2231.7540839176827
INFO:root:optimize_gain 0.9999992512342296
INFO:root:chosen var {'x_2_1', '

result path is /Users/aoyuzhan/Workplace/SolutionCenter/initiative/qc/hcls/drug-discovery/AWS-gcr-qc-life-science/source/src/molecule-unfolding/molecule-data/Aspirin_neal-sa_20220524-10.mol2, and result optimization file path is /Users/aoyuzhan/Workplace/SolutionCenter/initiative/qc/hcls/drug-discovery/AWS-gcr-qc-life-science/source/src/molecule-unfolding/molecule-data/Aspirin_neal-sa_20220524-10.json


In [55]:
sa_process_result.parameters

{'volume': {'optimize': 2231.755773173992,
  'initial': 2231.75575498,
  'gain': 1.0000000081523224,
  'unfolding_results': ['X_3_1', 'X_1_2', 'X_2_1'],
  'annealing_results': ['x_2_1',
   'x_3_1',
   'x_3_7',
   'x_3_4',
   'x_2_3',
   'x_3_3',
   'x_2_4',
   'x_2_8',
   'x_1_8',
   'x_1_4',
   'x_3_6',
   'x_2_2',
   'x_2_6',
   'x_3_5',
   'x_1_3',
   'x_3_8',
   'x_3_2',
   'x_2_5',
   'x_1_2',
   'x_1_7',
   'x_2_7',
   'x_1_1'],
  'optimize_info': {'optimize_state': True, 'result_rank': 9}}}

In the first block, we can see the **local time**
for SA is more than 150 seconds. 
With the **generate_optimize_pts()** method, the final 3D 
points after unfolding will be generated and saved as json file and mol2 files. The last 
block shows the optimizing results which are also stored in json files. 
It shows that the optimized result gains 
1.0006x increase in volume. The value for **unfolding_results** indicates 
that the rotatable bond 15 should rotate $270^o$ ($360/8*(7-1)$) and 
the rotatable bond 14 should rotate $0^o$ ($360/8*(1-1)$).
At the same time, you can run the post-process for QA:

In [322]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/f4081e34-c785-4ba7-9718-cbea1a308596/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/f4081e34-c785-4ba7-9718-cbea1a308596/results.json


time for dwave-qa: 
     local time is 10.384690284729004,
     task time is 1.969, 
     qpu total time is 0.117948, 
     qpu access time is 0.110488


we can see that there many types of time metrics for running QA.
This task has the **local time** of 5 s, which means the time between calling the api and 
getting the annealing result. The **task time** time is the metric from the json file in 
bucket. We can also see the **qpu total time** and **qpu access time** representing the 
actual time running in the QPU. Please refer to [Operation and Timing](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)
for details.

In [323]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_result_filepath, qa_result_json = qa_process_result.save_mol_file(f"{timestamp}")
print(f"result path is {qa_result_filepath}, and result optimization file path is {qa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'3': ['3'], '2': ['1']} var_dict_list [{'3': '3', '2': '1'}]
INFO:root:chosen var {'x_3_3', 'x_2_1'}
INFO:root:tor list {'X_3_3', 'X_2_1'}
INFO:root:initial 12.11393714296005
INFO:root:optimize 12.453192990579538
INFO:root:optimize_gain 1.0280054158789032
INFO:root:start physical check
INFO:root:fail at 3 to 17
INFO:root:physical check not pass!
INFO:root:var_dict_raw {'2': ['1'], '3': ['4']} var_dict_list [{'2': '1', '3': '4'}]
INFO:root:chosen var {'x_2_1', 'x_3_4'}
INFO:root:tor list {'X_3_4', 'X_2_1'}
INFO:root:initial 12.11393714296005
INFO:root:optimize 12.36453125062258
INFO:root:optimize_gain 1.0206864295814975
INFO:root:start physical check
INFO:root:save_mol_file 20220426-08
INFO:root:finish save /Users/aoyuzhan/Workplace/SolutionCenter/initiative/qc/hcls/drug-discovery/AWS-gcr-qc-life-science/source/src/molecule-unfolding/molecule-data/Aspirin_dwave-qa_20220426-08.mol2 and /Users/aoyuzhan/Workplace/SolutionCenter/init

result path is /Users/aoyuzhan/Workplace/SolutionCenter/initiative/qc/hcls/drug-discovery/AWS-gcr-qc-life-science/source/src/molecule-unfolding/molecule-data/Aspirin_dwave-qa_20220426-08.mol2, and result optimization file path is /Users/aoyuzhan/Workplace/SolutionCenter/initiative/qc/hcls/drug-discovery/AWS-gcr-qc-life-science/source/src/molecule-unfolding/molecule-data/Aspirin_dwave-qa_20220426-08.json


In [324]:
qa_process_result.parameters

{'volume': {'optimize': 12.36453125062258,
  'initial': 12.11393714296005,
  'gain': 1.0206864295814975,
  'unfolding_results': ['X_3_4', 'X_2_1'],
  'annealing_results': ['x_2_1', 'x_3_4'],
  'optimize_info': {'optimize_state': True, 'result_rank': 2}}}

In same way, the optimized results are translated the 3D points and saved 
as local json and mol2 files. The result indicates that QA gains 
1.0006x increase in 
volume.

Finally, We can open folders for the optimized results:

![optimize-results](../../../docs/en/images/optimize-results.png)

 <center>Optimize Results</center>

We can see the json result and mol2 file of SA and QA are 
stored in this place. If we carry out more 
experiments, more results with time stamp are 
stored incrementally. 
For visualization, we can use the following method to see the original the molecule and unfolding molecule.

In [37]:
# this shows the original molecule
qa_process_result.InteractView(raw_path, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal.mol2', description='mol'), IntSlider(value=800, de…

In [38]:
# copy the variable for the path of unfolding molecule, we can see the unfolding results. this also works for sa_process_result.
qa_process_result.InteractView(qa_result_filepath, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal_dwave-qa_20220323-14.mol2', description='mol'), In…

Alternatively, we can upload the mol2 file into 
[online viewer tool](https://www.rcsb.org/3d-view) 
to see the result:

![visual](../../../docs/en/images/visualization.png)

 <center>Visualization</center>

# Draft place

In [1]:
# result from symbolic annealing
# x_2_8 = 1, x_3_7 = 1
# result from pre-calc 
# x_2_1 = 1, x_3_4 = 1
# result from pre-calc, sum distance metrics
# x_2_1 = 1, x_3_2 = 1
import logging
from pyqubo import Array
from pyqubo import Num
from pyqubo import Binary
import math
import numpy as np

In [18]:
M=2
D=8

In [27]:
logging.info("!!!!symbolic method!!!!")
var = {} 
var_rb_map = {}
rb_var_map = {}

qmu_qubo._init_mol_file()
atom_pos_data = qmu_qubo.atom_pos_data

for m, name in enumerate(mol_data.bond_graph.rb_name):
    x_d = {}
    var_rb_map[str(m+1)] = name
    rb_var_map[str(name)] = str(m+1)
    d = 0
    exec (f"x_{m+1}_{d+1} = Binary(f'x_{m+1}_{d+1}')") 
    exec (f"x_{m+1}_{d+1} = 0") 
    exec (f"x_3_3 = 1") 
    # exec (f"x_3_7 = 1") 
    exec (f"x_2_1 = 1") 
    exec (f"x_d[str({d+1})] = x_{m+1}_{d+1}") 

    exec (f"s_value = math.sin(2*math.pi*{d}/({D}))")
    exec (f"x_{m+1}_s_1 = x_{m+1}_{d+1} * s_value")
    exec (f"s_value = math.cos(2*math.pi*{d}/({D}))")
    exec (f"x_{m+1}_c_1 = x_{m+1}_{d+1} * s_value")

    for d in range(1, D):
        exec (f"x_{m+1}_{d+1} = Binary(f'x_{m+1}_{d+1}')") 
        exec (f"x_{m+1}_{d+1} = 0") 
        exec (f"x_3_3 = 1") 
        # exec (f"x_2_8 = 1") 
        # exec (f"x_3_7 = 1") 
        exec (f"x_2_1 = 1") 
        exec (f"x_d[str({d+1})] = x_{m+1}_{d+1}") 

        exec (f"s_value = math.sin(2*math.pi*{d}/({D}))")
        exec (f"x_{m+1}_s_1 = x_{m+1}_s_1 + x_{m+1}_{d+1} * s_value")
        exec (f"s_value = math.cos(2*math.pi*{d}/({D}))")
        exec (f"x_{m+1}_c_1 = x_{m+1}_c_1 + x_{m+1}_{d+1} * s_value")
    exec(f"x_d['s_1'] = x_{m+1}_s_1")
    exec(f"x_d['c_1'] = x_{m+1}_c_1")
    var[str(m+1)] = x_d
test_hubo_distance = 0
def update_hubo(torsion_group, hubo_distance, tor_points):
    rot_mat = None

    for tor in torsion_group:
        c = var[rb_var_map[tor]]['c_1']
        s = var[rb_var_map[tor]]['s_1']
        start_pts = atom_pos_data[tor.split('+')[0]]
        end_pts = atom_pos_data[tor.split('+')[1]]
        logging.info(f"rot start pts {start_pts['pts']}, end pts {end_pts['pts']}")
        if rot_mat is None:
            rot_mat, mat_list = rot_mat_value(start_pts['pts'], end_pts['pts'], c, s)
        else:
            new_mat, mat_list = rot_mat_value(start_pts['pts'], end_pts['pts'], c, s)
            rot_mat = new_mat.matmul(rot_mat)
    # update points distance
    tor_points_f_0_set = tor_points['f_0_set']
    tor_points_f_1_set = tor_points['f_1_set']
    # tor_points_f_0_set = ['3']
    # tor_points_f_1_set = ['20']

    logging.debug(f"rot_mat is {rot_mat}")

    for f_0 in tor_points_f_0_set:
        for f_1 in tor_points_f_1_set:
            a_pos = atom_pos_data[f_0]['pts']
            b_pos = atom_pos_data[f_1]['pts']
            a_org_pos = Array([[a_pos[0]], [a_pos[1]], [a_pos[2]]])
            b_org_homo_pos = Array([[b_pos[0]], [b_pos[1]], [b_pos[2]], [1]])
            b_rot_homo_pos = rot_mat.matmul(b_org_homo_pos)
            b_rot_pos = Array([b_rot_homo_pos[0],b_rot_homo_pos[1],b_rot_homo_pos[2]])
            result_array = a_org_pos - b_rot_pos
            distance = result_array.T.matmul(result_array)
            logging.debug(f'b org {b_org_homo_pos}')
            logging.debug(f"a org pos {a_org_pos} b rotate {b_rot_pos} distance is {distance}")
            logging.info(f"pos f_0 {f_0}: {a_org_pos}")
            logging.info(f"pos f_1 {f_1}: original {b_org_homo_pos} rot {b_rot_pos}")
            logging.info(f"distance {distance}")
            hubo_distance = hubo_distance - distance[0][0]
    
    return hubo_distance


for ris in mol_data.bond_graph.sort_ris_data[str(M)].keys():
    start = time.time()
    logging.info(f"Rotables influence set(ris) group {ris}")
    end = time.time()
    torsion_group = ris.split(",")
    # if len(torsion_group) == 1:
    #     # update constraint
    #     hubo_constraints = update_constraint(ris, hubo_constraints)
    logging.info(f"torsion_group is {torsion_group}")

    ris_data = mol_data.bond_graph.sort_ris_data[str(M)][ris]

    test_hubo_distance = update_hubo(torsion_group, test_hubo_distance, ris_data)

INFO:root:!!!!symbolic method!!!!
INFO:root:Rotables influence set(ris) group 4+5
INFO:root:torsion_group is ['4+5']
INFO:root:rot start pts [-1.1519, -0.6914, 0.0093], end pts [-0.9578, -2.0342, 0.0079]
INFO:root:pos f_0 2: Array([[-1.3545],
       [-0.1059],
       [1.201]])
INFO:root:pos f_1 17: original Array([[1.1759],
       [-1.8858],
       [0.1003],
       [1]]) rot Array([[-1.026452255800657],
       [-2.206274499743402],
       [2.1407181460948657]])
INFO:root:distance Array([[5.402258555746593]])
INFO:root:pos f_0 2: Array([[-1.3545],
       [-0.1059],
       [1.201]])
INFO:root:pos f_1 6: original Array([[0.327],
       [-2.5523],
       [0.0594],
       [1]]) rot Array([[-0.9096590301343603],
       [-2.7322514894907117],
       [1.2046445278923124]])
INFO:root:distance Array([[7.09561891740464]])
INFO:root:pos f_0 2: Array([[-1.3545],
       [-0.1059],
       [1.201]])
INFO:root:pos f_1 12: original Array([[-4.3813],
       [-3.1301],
       [-0.1456],
       [1]]) rot A

In [28]:
# x_2_3 x_3_3
test_hubo_distance

-2290.909896238936

In [21]:
qmu_qubo._init_mol_file()
qmu_qubo.atom_pos_data['20']

{'pts': [-2.6833, -4.964, -0.0884],
 'idx': ([0, 0, 0], [0, 0, 0]),
 'vdw-radius': 1.2}

In [13]:
qmu_qubo._init_mol_file()

In [22]:
qmu_qubo.mol_data.atom_data['10']

{'atom_name': 'C10',
 'x': -2.0568,
 'y': -2.9052,
 'z': -0.0398,
 'atom_type': 'C.ar',
 'subst_id': 1,
 'subst_name': '<0>',
 'charge': -0.14,
 'vdw-radius': 1.7}

In [31]:
from utility.MolGeoCalc import *
pre_hubo_distance = {}
theta_option = [x * 360/D for x in range(D)]
# x_3_4
# f_0_set = ['1']
# f_1_set = ['10']

var = qmu_qubo.var

debug_complete_distance = 0

def _init_mol_file(raw_atom_data):
    atom_pos_data = {}
    for pt, info in raw_atom_data.items():
        atom_pos_data[pt] = {}
        atom_pos_data[pt]['pts'] = [info['x'], info['y'], info['z']]
        atom_pos_data[pt]['idx'] = ([0, 0, 0], [0, 0, 0])
        atom_pos_data[pt]['vdw-radius'] = info['vdw-radius']
    return atom_pos_data

def update_hubo(torsion_group, up_list, ris, debug_complete_distance):
    if len(torsion_group) == 1:
        for d in range(D):

            tor_list = up_list + \
                [var[rb_var_map[torsion_group[0]]][str(d+1)]]
            if tor_list != ['x_2_1'] and tor_list != ['x_3_3'] and tor_list != ['x_3_3','x_2_1']:
                continue
            # distance
            final_list_name = []
            if len(tor_list) == 1:
                final_list_name = tor_list + tor_list
            else:
                final_list_name = tor_list

            # update temp points and distance
            # TODO: !!!! temp disable for quick validation
            atom_pos_data = _init_mol_file(qmu_qubo.mol_data.atom_data)

            rb_set = mol_data.bond_graph.sort_ris_data[str(
                M)][ris]
            
            # rb_set['f_0_set'] = set()
            # rb_set['f_0_set'].add('3')
            # rb_set['f_1_set'] = set()
            # rb_set['f_1_set'].add('20')

            # build map for affected tor
            tor_map = {}
            tor_len = len(tor_list)
            logging.info(f"tor list {tor_list}")
            for base_idx in range(tor_len):
                tor_name = tor_list[base_idx]
                tor_map[tor_name] = set()
                base_rb_name = var_rb_map[tor_list[base_idx].split('_')[
                    1]]

                # get direction set
                direction_set = get_same_direction_set(
                    rb_set['f_1_set'], mol_data.bond_graph.rb_data, base_rb_name)

                for candi_idx in range(base_idx, tor_len):
                    candi_rb_name = var_rb_map[tor_list[candi_idx].split('_')[
                        1]].split('+')
                    for rb in candi_rb_name:
                        if rb in direction_set:
                            tor_map[tor_name].add(rb)

            # distance = update_pts_distance(
            #     atom_pos_data, rb_set, tor_map, var_rb_map, theta_option, True, True)
            def _gen_pts_pos_list(pt_set, atom_pos_data):
                return [atom_pos_data[pt]['pts'] for pt in pt_set]

            def _gen_pts_list(pt_set, atom_pos_data):
                return [atom_pos_data[pt] for pt in pt_set]

            def update_pts(start_pts, end_pts, pts_list, rotate_theta):
                rotate_list = []
                logging.debug("start_pts {}".format(start_pts))
                logging.debug("end_pts {}".format(end_pts))
                logging.debug("pts_list {}".format(pts_list))
                pi = 3.1415926
                for pt in pts_list:
                    if pt['idx'] != (start_pts[0]['pts'], end_pts[0]['pts']):
                        rotate_list.append(PointRotate3D(
                            start_pts[0]['pts'], end_pts[0]['pts'], pt['pts'], rotate_theta/180*pi))
                    else:
                        logging.debug("avoid same rotate *******")
                        rotate_list.append(pt['pts'])
                return rotate_list

            # rb_set
            for var_name, affect_tor_pts_set in tor_map.items():
                d = var_name.split('_')[2]
                rb_name = var_rb_map[var_name.split('_')[1]]
                # update points
                start_pts = atom_pos_data[rb_name.split('+')[0]]
                end_pts = atom_pos_data[rb_name.split('+')[1]]
                whole_set = set.union(rb_set['f_1_set'], affect_tor_pts_set)
                logging.debug(f"whole set is {whole_set}")
                gen_pts = _gen_pts_list(whole_set, atom_pos_data)
                logging.debug(f"gen pts is {gen_pts}")
                theta = theta_option[int(d)-1]
                rotate_list = update_pts([start_pts], [end_pts], gen_pts, theta)
                for pt_name, pt_value in zip(whole_set, rotate_list):
                    atom_pos_data[pt_name]['pts'] = pt_value

            distance = None
            # calculate distance
            logging.debug(f"pts 1: {atom_pos_data['1']}, pts 10: {atom_pos_data['10']}")
            distance = calc_distance_between_pts(_gen_pts_pos_list(
                rb_set['f_0_set'], atom_pos_data), _gen_pts_pos_list(rb_set['f_1_set'], atom_pos_data))
            # TODO: debug different calc methods
            all_distance_debug = 0
            all_distance_pow_debug = 0
            for f_0_idx in rb_set['f_0_set']:
                for f_1_idx in rb_set['f_1_set']:
                    f_0_pos = atom_pos_data[f_0_idx]['pts']
                    f_1_pos = atom_pos_data[f_1_idx]['pts']
                    sub_distance = np.linalg.norm(np.array(f_0_pos)-np.array(f_1_pos))
                    logging.info(f"pos f_0 {f_0_idx}: {f_0_pos}")
                    logging.info(f"pos f_1 {f_1_idx}: {f_1_pos}")
                    logging.info(f"distance {sub_distance**2}")
                    all_distance_debug = all_distance_debug + sub_distance
                    all_distance_pow_debug = all_distance_pow_debug + sub_distance**2

            pre_hubo_distance[tuple(final_list_name)] = -distance
            logging.info(
                f"final list {tor_list} with distance {distance} pow(2) {distance**2} debug {all_distance_debug} and {all_distance_pow_debug}")
            debug_complete_distance = debug_complete_distance + all_distance_pow_debug
    else:
        for d in range(D):
            final_list = up_list + \
                [var[rb_var_map[torsion_group[0]]][str(d+1)]]
            update_hubo(torsion_group[1:], final_list, ris, debug_complete_distance)


for ris in mol_data.bond_graph.sort_ris_data[str(M)].keys():
    start = time.time()
    logging.info(f"ris group {ris} ")
    end = time.time()
    torsion_group = ris.split(",")

    logging.info(torsion_group)
    # update hubo terms
    update_hubo(torsion_group, [], ris, debug_complete_distance)
    logging.debug(
        f"elapsed time for torsion group {ris} : {(end-start)/60} min")

INFO:root:ris group 4+5 
INFO:root:['4+5']
INFO:root:tor list ['x_3_3']
INFO:root:pos f_0 2: [-1.3545, -0.1059, 1.201]
INFO:root:pos f_1 17: [-1.0264521992351332, -2.2062744915695145, 2.1407181485750892]
INFO:root:distance 5.4022585631839535
INFO:root:pos f_0 2: [-1.3545, -0.1059, 1.201]
INFO:root:pos f_1 6: [-0.9096589983782766, -2.7322514849018695, 1.2046445292915071]
INFO:root:distance 7.09561892156383
INFO:root:pos f_0 2: [-1.3545, -0.1059, 1.201]
INFO:root:pos f_1 12: [-0.7218863100410442, -2.5975998158683895, -3.5377785934951036]
INFO:root:distance 29.064790611289485
INFO:root:pos f_0 2: [-1.3545, -0.1059, 1.201]
INFO:root:pos f_1 19: [-0.4059701172522536, -5.852369906898573, 0.015429446244366242]
INFO:root:distance 35.327202867288776
INFO:root:pos f_0 2: [-1.3545, -0.1059, 1.201]
INFO:root:pos f_1 10: [-0.8106503049117724, -2.723855707206223, -1.2051289620422614]
INFO:root:distance 12.938921157719765
INFO:root:pos f_0 2: [-1.3545, -0.1059, 1.201]
INFO:root:pos f_1 13: [-1.040122

In [32]:
238.24929209996776+72.52885708+1980.1317470436265

2290.9098962235944

In [ ]:
import numpy as np
from math import cos, sin

from utility.MolGeoCalc import *
r45_vector = (-1.1519, -0.6914, 0.0093)
r45_orig = (-0.9578, -2.0342, 0.0079)
# r45_vector = (1, 0, 0)
# r45_orig = (0, 0, 1)

# theta = math.pi/8 * 3
# c = cos(theta)
# s = sin(theta)
c = x_1_c
s = x_1_s
r45_rot_binary, r45_rot = rot_mat_value(r45_vector, r45_orig, c, s)

r24_vector = (-1.3545, -0.1059, 1.2020)
r24_orig = (-1.1519, -0.6914, 0.0093)
r24_rot_binary, r24_rot = rot_mat_value(r24_vector, r24_orig, c, s)

a = np.array([-2.4524, -0.3545, 0.0197, 1])

b = np.array([1.342, -2.343, 3.113, 1])

a_1 = np.matmul(np.matmul(r24_rot, r45_rot).round(4), a).round(4)

result_a_1 = (b - a_1)

distance = np.matmul(result_a_1.T, result_a_1)

# b = np.matmul(np.matmul(r24_rot, r45_rot).round(4), a).round(4)
# b = np.matmul(r24_rot, np.matmul(r45_rot, a))
# b = np.matmul(r24_rot, a)

# print(f"1st rot a_1 is {a_1} 2nd rot a_2 is {a_1}")
print(f"1st distance {distance}")
# a.shape
# r45_rot.shape
p2 = list(r45_vector)
p1 = list(r45_orig)
p0 = [a[0], a[1], a[2]]

a_1 = PointRotate3D(p1,p2,p0,theta)

p2 = list(r24_vector)
p1 = list(r24_orig)
p0 = [a_1[0], a_1[1], a_1[2]]

result_p = PointRotate3D(p1,p2,p0,theta)

b_list = [b[0], b[1], b[2]]

result_a_2 = []
for idx in range(3):
    result_a_2.append(b_list[idx] - result_p[idx])

distance_2 = 0

for v in result_a_2:
    distance_2 = distance_2 + v**2

# print(f"PointRotate3D 1st rot a_1 is {a_1} 2nd rot a_2 is {result_p}")
print(f"2nd distance {distance_2}")

In [24]:
import torch
import math
from torch import nn

In [26]:
n_qubits = 2
x = torch.rand((10, 1, 2**n_qubits))
x.shape

torch.Size([10, 1, 4])

In [35]:
np.random.uniform(0,1,1).tolist()

[0.1523620704438916]

In [32]:
from scipy.optimize import minimize
import numpy as np

opt_method = 'Powell'  # SLSQP, COBYLA, Nelder-Mead, BFGS, Powell, ...

def Bert_train(mask_rate):
    return 0

def Evaluate_probe_task(mask_rate):
    return 0

def objective_function(param):
    0
    100
    200
    300
    400
    # step0: input mask rate for next n steps
    # TODk
    next_mask_rate = forecast_model(param, past_prob_task, past_learning_metrics)
    # step1: input mask rate for next n steps
    # TODO: BERT_MODEL(param)
    Bert_train(next_mask_rate)
    # step2: output evaluate the training results
    # TODO: evaluate th
    Change_rate = Evaluate_probe_task()

    return -Change_rate

# set bounds for search space
model_initial_param = np.random.uniform(0, 1.0, 1).tolist()
bnds = [(0, 1.0)]


result = minimize(
    objective_function,
    mdoel_initial_param,
    method=opt_method,
    bounds=bnds,
)

torch.Size([2, 2, 2])